<a href="https://colab.research.google.com/github/ucfilho/Doubts_Students/blob/master/Age_Estimation_Final_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Idade

In [1]:
import zipfile
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Replace with your path!
!unzip -q /content/drive/MyDrive/TRY_ME/UTKFace.zip -d data


In [4]:
# !mv data/UTKFace data/UTKFace

In [5]:
'''
!gdown 1XqNOGo6QzeNYeOxsGIwBx-xlPT5_Cl0e

FILE='UTKFace.zip'

archive = zipfile.ZipFile(FILE, 'r')
archive.extractall()

img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)'''

'\n!gdown 1XqNOGo6QzeNYeOxsGIwBx-xlPT5_Cl0e\n\nFILE=\'UTKFace.zip\'\n\narchive = zipfile.ZipFile(FILE, \'r\')\narchive.extractall()\n\nimg_name=[]\nwith zipfile.ZipFile(FILE, "r") as f:\n    for name in f.namelist():\n        img_name.append(name)'

In [6]:
# para treino
!gdown 15dJJp8CJaaJ5JPoBgSlll_vCxy9vCdrV
df = pd.read_csv('train-utkface.csv')
df_train = df[['filename','age']]

# para val
!gdown 1l-DyeTfi5Q9tbZ9NgqGW0ABlr1X8kXqk
df = pd.read_csv('valid-utkface.csv')
df_val = df[['filename','age']]

# para test
!gdown 1c5W3OWc0fD8L3BAM9P5nNUlNGJHd8ZVv
df = pd.read_csv('test-utkface.csv')
df_test = df[['filename','age']]

# PATH_IMGS = '/sample_data'
PATH_IMGS = '/content/data'

Downloading...
From: https://drive.google.com/uc?id=15dJJp8CJaaJ5JPoBgSlll_vCxy9vCdrV
To: /content/train-utkface.csv
100% 740k/740k [00:00<00:00, 74.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1l-DyeTfi5Q9tbZ9NgqGW0ABlr1X8kXqk
To: /content/valid-utkface.csv
100% 185k/185k [00:00<00:00, 69.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1c5W3OWc0fD8L3BAM9P5nNUlNGJHd8ZVv
To: /content/test-utkface.csv
100% 231k/231k [00:00<00:00, 58.6MB/s]


In [ ]:
# PATH_IMGS contém o caminho para a pasta com as imagens
# image_size e target_size indicam o tamanho que a imagem será carregada, no exemplo, 128x128x3
# Para a regressão, o modelo tem class_mode igual a raw, já que usa os valores que estão na tabela
# A coluna age no dataframe tem a idade no formato inteiro
# Caso a memória estoure, vale a pena diminuir o batch_size

generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input)

train_generator = generator.flow_from_dataframe(
        df_train, directory=PATH_IMGS, x_col='filename', y_col=['age'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=True)

val_generator = generator.flow_from_dataframe(
        df_val, directory=PATH_IMGS, x_col='filename', y_col=['age'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=False)

test_generator = generator.flow_from_dataframe(
        df_test, directory=PATH_IMGS, x_col='filename', y_col=['age'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=False)


# Exemplo com ResNet50V2
backbone = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, input_shape=(128,128,3), pooling='avg')

for layer in backbone.layers:
  layer.trainable = True

model = tf.keras.Sequential([
    backbone,
    tf.keras.layers.Dense(1, activation=None)
])

model.compile(optimizer='adam', loss=['mae'])

model.fit(train_generator, epochs=10, validation_data=val_generator, batch_size=16)

Found 15170 validated image filenames.
Found 3793 validated image filenames.
Found 4742 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 2 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


94668760/94668760 [==============================] - 1s 0us/step
Epoch 1/10
 42/949 [>.............................] - ETA: 1:10:16 - loss: 16.7519

Gênero

In [ ]:
# PATH_IMGS contém o caminho para a pasta com as imagens
# image_size e target_size indicam o tamanho que a imagem será carregada, no exemplo, 128x128x3
# Para a classificação binária, o modelo tem class_mode igual a raw, já que usa os valores que estão na tabela (0 ou 1)
# A coluna gender no dataframe tem a idade no formato inteiro
# Caso a memória estoure, vale a pena diminuir o batch_size

generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input)

train_generator = generator.flow_from_dataframe(
        df_train, directory=PATH_IMGS, x_col='filename', y_col=['gender'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=True)

val_generator = generator.flow_from_dataframe(
        df_val, directory=PATH_IMGS, x_col='filename', y_col=['gender'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=False)

test_generator = generator.flow_from_dataframe(
        df_test, directory=PATH_IMGS, x_col='filename', y_col=['gender'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=False)


# Exemplo com ResNet50V2
backbone = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, input_shape=(128,128,3), pooling='avg')

for layer in backbone.layers:
  layer.trainable = True

model = tf.keras.Sequential([
    backbone,
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss=['binary_crossentropy'])

model.fit(train_generator, epochs=10, validation_data=val_generator, batch_size=16)

Raça

In [ ]:
# PATH_IMGS contém o caminho para a pasta com as imagens
# image_size e target_size indicam o tamanho que a imagem será carregada, no exemplo, 128x128x3
# Para a classificação multi-classe, o modelo pode usar class_mode igual a raw, porém adicionando a loss sparse_categorical_crossentropy, para tratar as classes como 0, 1, 2, 3, 4
# A coluna race no dataframe tem a idade no formato inteiro
# Caso a memória estoure, vale a pena diminuir o batch_size

generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input)

train_generator = generator.flow_from_dataframe(
        df_train, directory=PATH_IMGS, x_col='filename', y_col=['race'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=True)

val_generator = generator.flow_from_dataframe(
        df_val, directory=PATH_IMGS, x_col='filename', y_col=['race'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=False)

test_generator = generator.flow_from_dataframe(
        df_test, directory=PATH_IMGS, x_col='filename', y_col=['race'], image_size=(128, 128),
        target_size=(128, 128), class_mode='raw', batch_size=16, shuffle=False)


# Exemplo com ResNet50V2
backbone = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, input_shape=(128,128,3), pooling='avg')

for layer in backbone.layers:
  layer.trainable = True

model = tf.keras.Sequential([
    backbone,
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy'])

model.fit(train_generator, epochs=10, validation_data=val_generator, batch_size=16)